<a href="https://colab.research.google.com/github/Misha-private/Demo-repo/blob/main/ML_LAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ting's code for conversion betwee lon-lat coordinates and conformal Lambert coordinates

In [2]:
!pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00


In [3]:
import yaml
import numpy as np
from pyproj import Transformer
import netCDF4 as nc
import matplotlib.pyplot as plt
# Function to plot the domain using latitude and longitude
def plot_domain(lat, lon, title="Domain Plot"):
    plt.figure(figsize=(8, 6))
    plt.scatter(lon, lat, c='blue', marker='.', s=1)  # Plot the grid points
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(title)
    plt.grid(True)
    plt.savefig("NA-grid_domain_latlon.png")
# Function to plot the domain using Lambert Conformal Conic projection
def plot_lambert_domain(x, y, title="Domain Plot (Lambert Conformal Projection)"):
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, c='red', marker='.', s=1)  # Plot the grid points
    plt.xlabel('X (meters)')
    plt.ylabel('Y (meters)')
    plt.title(title)
    plt.grid(True)
    plt.savefig("NA-grid_domain_lambert_xy.png")
def convert_longitude_array(lon_array):
    # Convert longitude from 0-360 to -180-180
    lon_array = (lon_array + 180) % 360 - 180
    return lon_array

# Load the NetCDF file (replace 'your_file.nc' with your actual NetCDF file)
file_path = 'fv3_grid_spec'
dataset = nc.Dataset(file_path, mode='r')

# Read in grid_xt (longitude) and grid_yt (latitude)
lon = dataset.variables['grid_lont'][:]
lat = dataset.variables['grid_latt'][:]

lon=convert_longitude_array(lon)  #convert to -180 to 180
# Plot the domain using the latitude and longitude values
plot_domain(lat, lon, title="Grid Domain")
# Check the range of longitude and latitude
print(f"Longitude range: {lon.min()} to {lon.max()}")
print(f"Latitude range: {lat.min()} to {lat.max()}")

# Ensure that the longitude and latitude values are within the range for EPSG:4326
if lon.min() < -180 or lon.max() > 180:
    raise ValueError("Longitude values are out of range for EPSG:4326 (-180 to 180 degrees).")
if lat.min() < -90 or lat.max() > 90:
    raise ValueError("Latitude values are out of range for EPSG:4326 (-90 to 90 degrees).")

# Load the YAML projection setup (assuming it's in 'config.yaml')
with open('data_config.yaml', 'r') as file:
    yaml_data = yaml.safe_load(file)

# Access the projection parameters directly from the dictionary
central_longitude = yaml_data['projection']['kwargs']['central_longitude']
central_latitude = yaml_data['projection']['kwargs']['central_latitude']
standard_parallels = yaml_data['projection']['kwargs']['standard_parallels']

# Create the CRS (Coordinate Reference System) string based on the YAML configuration
mycrs = (f"+proj=lcc +lat_0={central_latitude} +lon_0={central_longitude} "
         f"+lat_1={standard_parallels[0]} +lat_2={standard_parallels[1]} "
         f"+x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")

# Set up the Transformer from WGS84 (lat/lon) to Lambert Conformal Conic projection
crstrans = Transformer.from_crs("EPSG:4326", mycrs, always_xy=True)

# Perform the coordinate transformation
x, y = crstrans.transform(lon, lat)
# Plot the domain in the Lambert Conformal Conic projection
plot_lambert_domain(x, y, title="Grid Domain (Lambert Conformal Conic Projection)")

# Combine the x and y coordinates into a single array
xy = np.array([x, y])  # Shape will be (2, n_lon, n_lat)

# Save the transformed coordinates to a .npy file
np.save("NA-grid_xy_coordinates.npy", xy)

print("Transformation complete. Coordinates saved to 'grid_xy_coordinates.npy'.")

# Close the NetCDF dataset
dataset.close()


FileNotFoundError: [Errno 2] No such file or directory: 'fv3_grid_spec'